In [1]:
!wget https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv

--2025-11-29 19:45:37--  https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv
Resolving www.fema.gov (www.fema.gov)... 184.29.213.174, 2600:1407:7400:b8d::1955, 2600:1407:7400:b8b::1955
Connecting to www.fema.gov (www.fema.gov)|184.29.213.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘DisasterDeclarationsSummaries.csv’

DisasterDeclaration     [   <=>              ]  21.54M  38.8MB/s    in 0.6s    

2025-11-29 19:45:38 (38.8 MB/s) - ‘DisasterDeclarationsSummaries.csv’ saved [22586640]



In [3]:
!unzip us-state-boundaries.zip -d us-state-boundaries

Archive:  us-state-boundaries.zip
   creating: us-state-boundaries/us-state-boundaries/
  inflating: us-state-boundaries/us-state-boundaries/us-state-boundaries.prj  
  inflating: us-state-boundaries/us-state-boundaries/us-state-boundaries.shp  
  inflating: us-state-boundaries/us-state-boundaries/us-state-boundaries.shx  
  inflating: us-state-boundaries/us-state-boundaries/us-state-boundaries.dbf  
 extracting: us-state-boundaries/us-state-boundaries/us-state-boundaries.cpg  


In [4]:
import pandas as pd
import geopandas as gpd

In [6]:
data = pd.read_csv('DisasterDeclarationsSummaries.csv')
geodata = gpd.read_file('us-state-boundaries/us-state-boundaries/us-state-boundaries.shp')

/tmp/ipython-input-264231089.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('DisasterDeclarationsSummaries.csv')


In [7]:

dates = pd.to_datetime(data["declarationDate"],format="ISO8601")

startdate = str(dates.min().date())
enddate = str(dates.max().date())

In [8]:
scores = data["state"].value_counts().reset_index().copy()
scores["score"] = scores["count"] / scores["count"].sum()
scores.drop("count", axis=1, inplace=True)

In [9]:
state_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "American Samoa": "AS",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "District of Columbia": "DC",
    "Florida": "FL",
    "Georgia": "GA",
    "Guam": "GU",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Northern Mariana Islands": "MP",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Puerto Rico": "PR",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "United States Virgin Islands": "VI",
    "Commonwealth of the Northern Mariana Islands": "MP"
}


geodata["code"] = geodata["name"].map(state_codes)

In [10]:
geometry_data = geodata[["code","geometry"]]

In [11]:
final_data = scores.merge(geometry_data, left_on="state", right_on="code")

In [12]:
final_data

,state,score,code,geometry
0,TX,0.078617,TX,"POLYGON ((-103.98018 32.00012, -103.06451 32.0..."
1,KY,0.047220,KY,"MULTIPOLYGON (((-88.06133 37.50533, -88.06297 ..."
2,MO,0.041308,MO,"POLYGON ((-94.60803 38.84721, -94.60713 39.100..."
3,FL,0.040739,FL,"MULTIPOLYGON (((-86.38865 30.99418, -85.99861 ..."
4,GA,0.038725,GA,"POLYGON ((-85.38658 33.90172, -85.42947 34.125..."
5,VA,0.038258,VA,"POLYGON ((-78.86928 38.76299, -78.84981 38.793..."
6,LA,0.037791,LA,"POLYGON ((-92.06911 33.00816, -91.50012 33.006..."
7,OK,0.037674,OK,"POLYGON ((-100.00376 36.49985, -103.00237 36.5..."
8,NC,0.033996,NC,"POLYGON ((-82.41674 36.07283, -82.4098 36.0830..."
9,PR,0.030886,PR,"MULTIPOLYGON (((-67.20794 18.44228, -67.21713 ..."


In [13]:
final_data.drop("code", axis=1,inplace=True)

In [14]:
final_data = gpd.GeoDataFrame(final_data,geometry="geometry",crs="epsg:4326")

In [15]:
final_data.to_file("states_scores.gpkg",driver="GPKG")

## Create 10 by 10 km Hexgones using qgis

In [17]:
data = gpd.read_file("hexagones.gpkg")

# Create The Token from [here](https://www.ncei.noaa.gov/cdo-web/token)

In [18]:
!wget https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

--2025-11-29 19:57:39--  https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
Resolving www.ncei.noaa.gov (www.ncei.noaa.gov)... 205.167.25.178, 205.167.25.171, 205.167.25.167, ...
Connecting to www.ncei.noaa.gov (www.ncei.noaa.gov)|205.167.25.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11150588 (11M) [text/plain]
Saving to: ‘ghcnd-stations.txt’

ghcnd-stations.txt  100%[===================>]  10.63M  26.2MB/s    in 0.4s    

2025-11-29 19:57:39 (26.2 MB/s) - ‘ghcnd-stations.txt’ saved [11150588/11150588]



In [19]:
stations = pd.read_fwf(
    "ghcnd-stations.txt",
    colspecs=[(0,11),(12,20),(21,30),(31,37),(41,71)],
    names=["station_id","latitude","longitude","elevation","name"])
stations.head()

,station_id,latitude,longitude,elevation,name
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL


In [53]:
station.shape

(129658, 6)

In [20]:
station = gpd.GeoDataFrame(stations,geometry=gpd.points_from_xy(stations["longitude"],stations["latitude"]),crs="epsg:4326")
station.to_file("stations.gpkg",driver="GPKG")

## join stations to hexgones with threshhold of 50km and max station eq 5 and drop unmatched hexagons

In [22]:
!unzip hex_data.zip

Archive:  hex_data.zip
   creating: hex_data/
  inflating: hex_data/hexagons_v2.gpkg  


In [165]:
hex_data = gpd.read_file("selected_hex_v2.gpkg")

In [166]:
hex_data.head()

,left,top,right,bottom,state,score,station_id,elevation,name,class,n,distance,feature_x,feature_y,nearest_x,nearest_y,geometry
0,-178.105253,51.790687,-177.989783,51.690687,AK,0.005065,USC00508995,44.2,TANAGA ISLAND,A,1,1.307894,-178.018650,51.690687,-177.970216,50.383690,"MULTIPOLYGON (((-178.07639 51.79069, -178.0186..."
1,-176.719612,51.890687,-176.604142,51.790687,AK,0.005065,USW00025704,5.2,ADAK AP,A,1,1.319150,-176.633010,51.790687,-176.543029,50.474609,"MULTIPOLYGON (((-176.69074 51.89069, -176.6330..."
2,-174.294741,52.290687,-174.179271,52.190687,AK,0.005065,USC00500433,11.9,ATKA,A,1,1.487620,-174.208139,52.190687,-174.065216,50.709949,"MULTIPOLYGON (((-174.26587 52.29069, -174.2081..."
3,-171.783268,63.840687,-171.667797,63.740687,AK,0.005065,USW00026703,8.2,GAMBELL,A,1,6.009692,-171.667797,63.790687,-165.781021,64.999929,"MULTIPOLYGON (((-171.75642 63.83718, -171.7146..."
4,-171.177050,-11.009313,-171.061580,-11.109313,AS,0.001109,AQC00914822,3.0,SWAIN ISLAND,A,1,33.636376,-171.090447,-11.009313,-155.691190,18.895012,"MULTIPOLYGON (((-171.09045 -11.00931, -171.061..."


In [167]:
stations = hex_data["station_id"].apply(lambda f:f"{f}.csv.gz")
stations_txt = stations.to_list()

In [168]:
len(stations_txt)

77372

In [173]:
!mkdir ghcn_usa

In [174]:
import requests
from tqdm import tqdm

base = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_station/"

for sid in tqdm(stations_txt[:5000]):
    url = base + sid
    r = requests.get(url)
    open(f"./ghcn_usa/{sid}", "wb").write(r.content)


100%|██████████| 5000/5000 [17:46<00:00,  4.69it/s]


In [175]:
!gunzip ghcn_usa/*.gz

In [176]:
sample = stations_txt[:5000]

# All possible elements
elements = ["PRCP", "SNOW", "SNWD", "TMAX", "TMIN", "TAVG", "DAPR", "MDPR"]

# List to collect rows
records = []

cols = ["ID", "DATE", "ELEMENT", "VALUE", "MFLAG", "QFLAG", "SFLAG", "OBS_TIME"]

# Measurement flag weights
flag_weights = {
    "H": 0.95,
    "P": 1.0,
    "R": 1.0,
    "T": 0.0,
    np.nan: 1.0
}

# Date filter
startdate = "1960-01-01"
enddate = "2014-12-31"

for sid in tqdm(sample):
    sid = sid.replace(".gz", "")

    # Read CSV
    df = pd.read_csv(
        f"./ghcn_usa/{sid}",
        header=None,
        names=cols,
        dtype={"ID": str, "DATE": str, "ELEMENT": str},
        low_memory=False
    )

    station_id = df["ID"].iloc[0]
    df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

    # Apply MFLAG weights
    df["VALUE"] = df["VALUE"] * df["MFLAG"].map(flag_weights).fillna(1.0)

    # Pivot to wide format
    df_wide = df.pivot_table(index="DATE", columns="ELEMENT", values="VALUE", fill_value=0)
    df_wide = df_wide.reset_index()

    # Filter by date
    df_wide = df_wide.loc[df_wide["DATE"].between(startdate, enddate)].copy()

    # Convert units for temperatures and precipitation
    for col in df_wide.columns:
        if col in ["TMAX", "TMIN", "TAVG"]:
            df_wide[col] = df_wide[col] / 10
        if col in ["PRCP"]:
            df_wide[col] = df_wide[col] / 10
        # SNOW and SNWD are already in mm, leave as is

    # Compute per-station averages safely
    agg = {"STATION_ID": station_id}
    for col in elements:
        agg[col] = df_wide.get(col, pd.Series(0)).mean()

    records.append(agg)

# Final DataFrame
new_df = pd.DataFrame(records)
new_df.head()


100%|██████████| 5000/5000 [04:37<00:00, 18.02it/s]


,STATION_ID,PRCP,SNOW,SNWD,TMAX,TMIN,TAVG,DAPR,MDPR
0,USC00508995,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000
1,USW00025704,3.767006,7.031625,21.612665,7.163763,2.305490,0.0,0.012299,0.360761
2,USC00500433,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000
3,USW00026703,0.950921,1.622594,75.576151,-1.338912,-5.479205,0.0,0.000837,0.019247
4,AQC00914822,7.457924,0.000000,0.000000,30.811719,21.856362,0.0,0.004464,0.790179


In [177]:
new_df.fillna(0,inplace=True)

In [178]:
new_df.head()

,STATION_ID,PRCP,SNOW,SNWD,TMAX,TMIN,TAVG,DAPR,MDPR
0,USC00508995,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,USW00025704,3.767006,7.031625,21.612665,7.163763,2.305490,0.0,0.012299,0.360761
2,USC00500433,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,USW00026703,0.950921,1.622594,75.576151,-1.338912,-5.479205,0.0,0.000837,0.019247
4,AQC00914822,7.457924,0.000000,0.000000,30.811719,21.856362,0.0,0.004464,0.790179


In [215]:
new_hex_data = hex_data.merge(new_df,left_on="station_id",right_on="STATION_ID",how="left").dropna().copy()

In [216]:
new_hex_data.head()

,left,top,right,bottom,state,score,station_id,elevation,name,class,...,geometry,STATION_ID,PRCP,SNOW,SNWD,TMAX,TMIN,TAVG,DAPR,MDPR
0,-178.105253,51.790687,-177.989783,51.690687,AK,0.005065,USC00508995,44.2,TANAGA ISLAND,A,...,"MULTIPOLYGON (((-178.07639 51.79069, -178.0186...",USC00508995,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,-176.719612,51.890687,-176.604142,51.790687,AK,0.005065,USW00025704,5.2,ADAK AP,A,...,"MULTIPOLYGON (((-176.69074 51.89069, -176.6330...",USW00025704,3.767006,7.031625,21.612665,7.163763,2.305490,0.0,0.012299,0.360761
2,-174.294741,52.290687,-174.179271,52.190687,AK,0.005065,USC00500433,11.9,ATKA,A,...,"MULTIPOLYGON (((-174.26587 52.29069, -174.2081...",USC00500433,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,-171.783268,63.840687,-171.667797,63.740687,AK,0.005065,USW00026703,8.2,GAMBELL,A,...,"MULTIPOLYGON (((-171.75642 63.83718, -171.7146...",USW00026703,0.950921,1.622594,75.576151,-1.338912,-5.479205,0.0,0.000837,0.019247
4,-171.177050,-11.009313,-171.061580,-11.109313,AS,0.001109,AQC00914822,3.0,SWAIN ISLAND,A,...,"MULTIPOLYGON (((-171.09045 -11.00931, -171.061...",AQC00914822,7.457924,0.000000,0.000000,30.811719,21.856362,0.0,0.004464,0.790179


In [217]:
new_hex_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 184316 entries, 0 to 185636
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   left        184316 non-null  float64 
 1   top         184316 non-null  float64 
 2   right       184316 non-null  float64 
 3   bottom      184316 non-null  float64 
 4   state       184316 non-null  object  
 5   score       184316 non-null  float64 
 6   station_id  184316 non-null  object  
 7   elevation   184316 non-null  float64 
 8   name        184316 non-null  object  
 9   class       184316 non-null  object  
 10  n           184316 non-null  int32   
 11  distance    184316 non-null  float64 
 12  feature_x   184316 non-null  float64 
 13  feature_y   184316 non-null  float64 
 14  nearest_x   184316 non-null  float64 
 15  nearest_y   184316 non-null  float64 
 16  geometry    184316 non-null  geometry
 17  STATION_ID  184316 non-null  object  
 18  PRCP        184316 no

In [218]:
new_hex_data = new_hex_data.drop(columns=["name","left","right","top","bottom","class","feature_x","feature_y","nearest_x","nearest_y","geometry","STATION_ID","n","station_id"])

In [219]:
new_hex_data.head()

,state,score,elevation,distance,PRCP,SNOW,SNWD,TMAX,TMIN,TAVG,DAPR,MDPR
0,AK,0.005065,44.2,1.307894,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,AK,0.005065,5.2,1.319150,3.767006,7.031625,21.612665,7.163763,2.305490,0.0,0.012299,0.360761
2,AK,0.005065,11.9,1.487620,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,AK,0.005065,8.2,6.009692,0.950921,1.622594,75.576151,-1.338912,-5.479205,0.0,0.000837,0.019247
4,AS,0.001109,3.0,33.636376,7.457924,0.000000,0.000000,30.811719,21.856362,0.0,0.004464,0.790179


In [222]:
new_hex_data.iloc[1].to_dict()

{'state': 'AK',
 'score': 0.005065027952531784,
 'elevation': 5.2,
 'distance': 1.3191503128723645,
 'PRCP': 3.767005856515373,
 'SNOW': 7.031625183016105,
 'SNWD': 21.612664714494876,
 'TMAX': 7.163762811127379,
 'TMIN': 2.3054904831625187,
 'TAVG': 0.0,
 'DAPR': 0.012298682284040996,
 'MDPR': 0.36076134699853585}

In [220]:
new_hex_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184316 entries, 0 to 185636
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   state      184316 non-null  object 
 1   score      184316 non-null  float64
 2   elevation  184316 non-null  float64
 3   distance   184316 non-null  float64
 4   PRCP       184316 non-null  float64
 5   SNOW       184316 non-null  float64
 6   SNWD       184316 non-null  float64
 7   TMAX       184316 non-null  float64
 8   TMIN       184316 non-null  float64
 9   TAVG       184316 non-null  float64
 10  DAPR       184316 non-null  float64
 11  MDPR       184316 non-null  float64
dtypes: float64(11), object(1)
memory usage: 18.3+ MB


In [209]:
from sklearn.preprocessing  import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error,r2_score
from xgboost import  XGBRegressor
from sklearn.model_selection import train_test_split

In [210]:
le = LabelEncoder()
new_hex_data["state"] = le.fit_transform(new_hex_data["state"])

y = new_hex_data["score"]
X = new_hex_data.drop("score",axis=1)

In [211]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [212]:
model = XGBRegressor()
model.fit(x_train,y_train)

pred = model.predict(x_test)

print(f"MAPE: {mean_absolute_percentage_error(y_test,pred)}")
print(f"MAE: {mean_absolute_error(y_test,pred)}")
print(f"R2: {r2_score(y_test,pred)}")

MAPE: 0.0009377378117646942
MAE: 3.077242165786185e-06
R2: 0.9999998770337482


In [213]:
import pickle

filename = 'xgboost.pkl'
with open(filename, 'wb') as f:
    pickle.dump(model, f)

In [214]:
class_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
class_mapping

{'AK': np.int64(0),
 'AS': np.int64(1),
 'CA': np.int64(2),
 'HI': np.int64(3),
 'OR': np.int64(4),
 'WA': np.int64(5)}